---
# SVM - Caso Titanic

En este notebook, utilizaremos SVM para entrenar un algoritmo de clasificación que permita establecer un modelo que prediga los sobrevivientes del dataset Titanic

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Carga de Datos y Análisis Exploratorio

In [5]:
df = pd.read_csv('titanic.csv')

In [6]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


### Valores perdidos

La cabina no aporta mucho en el análisis, pero la edad es una variable importante para predecir si una persona sobrevivió o no a la trajedia. A continuación, una forma de "visualizar" los valores perdidos.

Haremos una imputación de la edad del pasajero en función del promedio de edad de la clase en la que viajaba.

In [7]:
edades = round(df.groupby('Pclass')['Age'].mean())
edades

,Age
Pclass,
1,38.0
2,30.0
3,25.0


In [8]:
# creamos una funcion que imputa la edad en caso de estar nula
def imputar_edad(cols):
    edad = cols[0]
    pclass = cols[1]
    if(pd.isnull(edad)):
        if(pclass==1):
            return 38
        if(pclass==2):
            return 30
        if(pclass==3):
            return 25
    else:
        return edad

In [9]:
df['Age'] = df[['Age','Pclass']].apply(imputar_edad, axis=1)

/tmp/ipython-input-3628055789.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edad = cols[0]
/tmp/ipython-input-3628055789.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pclass = cols[1]


In [10]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


### Formulación del modelo

En esta sección seleccionaremos las variables que incorporaremos en la matriz de predictores "X" y en el vector de resultados "y", ya que no todas las columnas son de utilidad.

In [11]:
# Seleccionamos variables predictoras y variable objetivo
df_model = df.copy()
df_model['Embarked'].fillna(df_model['Embarked'].mode()[0], inplace=True)

feature_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df_model[feature_cols]
y = df_model['Survived']

# Convertimos variables categóricas a dummies
X = pd.get_dummies(X, drop_first=True)

X.head()

/tmp/ipython-input-2731026035.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['Embarked'].fillna(df_model['Embarked'].mode()[0], inplace=True)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,True,False,True
1,1,38.0,1,0,71.2833,False,False,False
2,3,26.0,0,0,7.9250,False,False,True
3,1,35.0,1,0,53.1000,False,False,True
4,3,35.0,0,0,8.0500,True,False,True


### Validación Cruzada

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    random_state=0,
                                                    stratify=y)

### Modelamiento con SVM kernel lineal

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Escalamos los datos (muy importante para SVM)
scaler_linear = StandardScaler()
X_train_scaled_lin = scaler_linear.fit_transform(X_train)
X_test_scaled_lin = scaler_linear.transform(X_test)

# Modelo SVM con kernel lineal
svm_linear = SVC(kernel='linear', C=1.0, random_state=0)
svm_linear.fit(X_train_scaled_lin, y_train)

y_pred_linear = svm_linear.predict(X_test_scaled_lin)

In [15]:
print("Resultados SVM kernel lineal")
print("Accuracy:", accuracy_score(y_test, y_pred_linear))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_linear))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_linear))

Resultados SVM kernel lineal
Accuracy: 0.776536312849162
Matriz de confusión:
 [[93 17]
 [23 46]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.85      0.82       110
           1       0.73      0.67      0.70        69

    accuracy                           0.78       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.77      0.78      0.77       179



### Modelamiento con SVM kernel rgf

In [16]:
# Modelo SVM con kernel RBF (radial)
scaler_rbf = StandardScaler()
X_train_scaled_rbf = scaler_rbf.fit_transform(X_train)
X_test_scaled_rbf = scaler_rbf.transform(X_test)

svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=0)
svm_rbf.fit(X_train_scaled_rbf, y_train)

y_pred_rbf = svm_rbf.predict(X_test_scaled_rbf)

In [17]:
print("Resultados SVM kernel RBF")
print("Accuracy:", accuracy_score(y_test, y_pred_rbf))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_rbf))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_rbf))

Resultados SVM kernel RBF
Accuracy: 0.8044692737430168
Matriz de confusión:
 [[99 11]
 [24 45]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.90      0.85       110
           1       0.80      0.65      0.72        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.78       179
weighted avg       0.80      0.80      0.80       179



### Optimice los parámetros Gamma, C (regularización), y Kernel

Para esto, utilice una grilla de búsqueda exhaustiva (GridSearchCV). Tome como referencia la siguiente documentación:

https://scikit-learn.org/stable/modules/grid_search.html

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Pipeline: escalado + SVM
pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# Definimos grilla de búsqueda
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.01, 0.1, 1, 'scale'],
    'svc__kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(estimator=pipe_svm,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

In [20]:
# Entrenamos la búsqueda en grilla
grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:")
print(grid_search.best_params_)
print("Mejor accuracy en validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

print("\nResultados mejor modelo (GridSearchCV)")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_best))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_best))

KeyboardInterrupt: 

# Análisis del impacto de GridSearchCV en el Modelo SVM

**1. Modelo Base**

Al utilizar los parámetros por defecto, el modelo establece una frontera de decisión genérica. Aunque logra un desempeño aceptable gracias al escalado previo de los datos (StandardScaler), corre el riesgo de no capturar los patrones complejos del dataset o, por el contrario, sobreajustarse a valores atípicos, ya que su penalización por error no está calibrada para el caso del Titanic.



**2. Modelo Optimizado**

Al implementar la búsqueda en grilla, el algoritmo prueba múltiples combinaciones de hiperparámetros ($C$, gamma, kernel) y las evalúa usando Validación Cruzada (5-fold). Esto obliga al modelo a adaptarse a la verdadera estructura de los datos.

# Principales conclusiones

El modelo optimizado ajusta su margen de error (parámetro $C$), lo que incrementa su precisión y lo hace mucho más robusto frente a datos nuevos.

Por otra parte, la optimización fue exitosa porque los datos se estandarizaron primero. Sin esto, variables con rangos amplios como la tarifa (Fare) habrían dominado a variables más pequeñas como la edad (Age).

Finalmente, aunque probar todas las combinaciones exige más tiempo de procesamiento, el salto en la calidad predictiva y la prevención del sobreajuste justifican plenamente el uso de GridSearchCV.


---

### `param_grid`

Es un diccionario que dice **qué hiperparámetros probar** y **con qué valores**.

- **`'svc__C'`**  
  - `C` es el hiperparámetro de **regularización** de SVM.  
  - Controla cuánto permite el modelo equivocarse en el entrenamiento:  
    - C pequeño → más regularización → margen más ancho, más sesgo, menos sobreajuste.  
    - C grande → menos regularización → se ajusta más a los datos, más riesgo de sobreajuste.  
  - El prefijo `svc__` indica que este parámetro pertenece al paso `'svc'` del `Pipeline` (`('svc', SVC())`).

- **`'svc__gamma'`**  
  - `gamma` controla la **influencia de cada punto de entrenamiento** en kernels no lineales (`rbf`, `poly`).  
  - Valores:
    - Pequeño (0.01) → efecto más suave, decision boundary más “suave”.  
    - Grande (1) → frontera muy curva, riesgo de sobreajuste.  
    - `'scale'` → valor automático recomendado por scikit‑learn.  
  - También se refiere al `SVC` dentro del pipeline (`svc__`).

- **`'svc__kernel'`**  
  - Tipo de **kernel** (la forma de la frontera de decisión):
    - `'linear'`: frontera lineal (hiperplano).  
    - `'rbf'`: frontera no lineal de tipo gaussiano.  
    - `'poly'`: frontera polinómica.  
  - Otra vez, `svc__` porque es un parámetro del `SVC` del pipeline.

---

### `GridSearchCV(...)`

- **`estimator=pipe_svm`**  
  El modelo completo que se va a probar en la grilla: `StandardScaler` + `SVC`.

- **`param_grid=param_grid`**  
  La grilla de combinaciones de hiperparámetros que definiste arriba.

- **`cv=5`**  
  Usa **validación cruzada 5‑fold**: divide los datos de entrenamiento en 5 partes, entrena en 4 y valida en 1, rotando.

- **`scoring='accuracy'`**  
  Métrica que se optimiza: el mejor modelo será el que tenga **mayor accuracy promedio** en la validación cruzada.

- **`n_jobs=-1`**  
  Usa **todos los núcleos disponibles** de la CPU para paralelizar las combinaciones de la grilla (más rápido).